In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
import os

# Configuration

In [2]:
data_path = '/kaggle/input/file-csv/File CSV/MIT-BIH/'
window_input= 200
window_out= 200
train_size = 0.8
test_size = 1 - train_size
data_set = {
  0: "testset",
  1: "trainset"
}

# Start

## DỰ ĐOÁN TÍN HIỆU ECG

In [3]:
def get_data(istrainset):    
    missing_file = 0
    total_sample = 0
    directory = f"{data_path}{data_set[istrainset]}/"
    X, y = [], []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            df = pd.read_csv(f, header=None)
            data=df.drop(columns=187)
            data=data.values
            # Số lượng lặp qua dữ liệu
            num_samples = len(data) - window_input - window_out 
            # print("----------------------")
            # print("🍒Len data:", len(data))
            # print("🍒Num_samples:", num_samples)
            # Đối với tập train
            # Tạo dữ liệu train từ cửa sổ trượt
            if(num_samples>0):
                total_sample = total_sample + num_samples
                for i in range(num_samples):
                    X_window = data[i:i+window_input]
                    y_window = data[i+window_input+window_out:i+window_input+window_out+1]

                    X.append(X_window)
                    y.append(y_window)
            else:
                missing_file = missing_file + 1
    print(f"Num of file can not use due to its missing of length: {missing_file}")
    print(f"Number of sample: {len(y)}/{len(X)}/{total_sample}")
    return X,y

In [4]:
X_train, y_train = get_data(1)
X_test, y_test = get_data(0)

Num of file can not use due to its missing of length: 10
Number of sample: 91632/91632/91632
Num of file can not use due to its missing of length: 2
Number of sample: 26080/26080/26080


In [5]:
print(f"Len(X_train/y_train): {len(X_train)}/{len(y_train)}")
print(f"Len(X_test/y_test): {len(X_test)}/{len(y_test)}")
print(f"Test Rate: ",len(X_test)/(len(X_train)+len(X_test)))
print(f"Train Rate: ",len(X_train)/(len(X_train)+len(X_test)))

Len(X_train/y_train): 91632/91632
Len(X_test/y_test): 26080/26080
Test Rate:  0.22155770014951748
Train Rate:  0.7784422998504825


In [6]:
# X_train= np.array(X_train)
# y_train= np.concatenate(y_train)
# X_test= np.array(X_test)
# y_test= np.concatenate(y_test)

In [7]:
# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

In [8]:
class Dataset:
    def __init__(self, data, label):
        self.data = np.array(data)
        self.label = np.concatenate(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        # read data
        X = self.data[i]
        y = self.label[i]
        return X, y

In [9]:
class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size,size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size= size

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)

    def __len__(self):
        return self.size //self.batch_size

In [10]:
train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)

In [ ]:
train_loader = Dataloader(train_dataset, 16,len(train_dataset))
test_loader = Dataloader(test_dataset,16,len(test_dataset))

In [ ]:
# Tạo một mô hình LSTM
model = Sequential()
model.add(LSTM(64,activation='relu' ,input_shape=(40, 187)))  # Đặt input_shape phù hợp với kích thước của mảng X_train
model.add(Dense(187))# Đặt lớp Dense phù hợp với kích thước của mảng y_train
# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit_generator(train_loader, validation_data=test_loader, epochs=5, verbose=1)

In [ ]:
# Dự đoán trên tập kiểm tra
y_pred = model.predict(test_dataset[:][0])

### Đánh giá độ chính xác mô hình

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Độ đo Mean Squared Error (MSE)
mse = mean_squared_error(test_dataset[:][1], y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Độ đo R-squared (R2)
r2 = r2_score(test_dataset[:][1], y_pred)
print(f"R-squared (R2): {r2}")

Trực quan 

In [ ]:
import matplotlib.pyplot as plt

# Chọn một ví dụ cụ thể để trực quan hóa
example_index = 40  # Thay đổi chỉ số ví dụ nếu cần

# Lấy giá trị dự đoán và giá trị thực tế tương ứng
y_pred_example = y_pred[example_index]
y_test_example = test_dataset[:][1][example_index]

# Vẽ biểu đồ cho giá trị dự đoán (màu xanh) và giá trị thực tế (màu đỏ)
plt.figure(figsize=(12, 6))
plt.plot(y_pred_example, label='Dự đoán', color='blue')
plt.plot(y_test_example, label='Thực tế', color='red')
plt.xlabel('Cột giá trị')
plt.ylabel('Giá trị')
plt.title('So sánh Dự đoán và Thực tế cho ví dụ cụ thể')
plt.legend()
plt.grid()
plt.show()


#### Giải phóng dữ liệu

In [ ]:
# import gc
# # Giải phóng dữ liệu đào tạo và kiểm tra
# X_class_train=None
# # Sử dụng garbage collection để giải phóng bộ nhớ
# gc.collect()

## PHÂN LỚP TÍN HIỆU ECG TƯƠNG LAI

### Preprocesing

In [ ]:
def get_data_class(istrainset):    
    missing_file = 0
    total_sample = 0
    directory = f"{data_path}{data_set[istrainset]}/"
    X, y = [], []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            df = pd.read_csv(f, header=None)
            data=df.values
            X_window = data[:,:-1]
            y_window = data[:,-1]
            X.append(X_window)
            y.append(y_window)
    print(f"Number of sample: {len(y)}/{len(X)}")
    return X,y

In [ ]:
X_class_train, y_class_train = get_data_class(1)

In [ ]:
X_class_train= np.concatenate(X_class_train,axis=0)

In [ ]:
y_class_train= np.concatenate(y_class_train, axis=0)
y_class_train=y_class_train.astype(int)

In [ ]:
#ECG dự đoán phân lớp X_class_test
X_class_test= y_pred
X_class_test

In [ ]:
#Lây lớp của y_test
def get_data_class2(istrainset):    
    missing_file = 0
    total_sample = 0
    directory = f"{data_path}{data_set[istrainset]}/"
    X, y = [], []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            df = pd.read_csv(f, header=None)
            data=df.values
            y_window = data[window_input+window_out:,-1]
            y.append(y_window)
    print(f"Number of sample: {len(y)}")
    return y

In [ ]:
y_class= get_data_class2(0)

In [ ]:
#Xác định y_test_class ban đầu
y_class=np.concatenate(y_class)
y_class_test=y_class.astype(int)

In [ ]:
print(X_class_train.shape)
print(y_class_train.shape)
print(X_class_test.shape)
print(y_class_test.shape)

In [ ]:
# import numpy as np

# unique_values, counts = np.unique(y_class_test, return_counts=True)

# for value, count in zip(unique_values, counts):
#     print(f"{value}: {count}")

### Mô hình đơn

#### Mô hình LSTM

In [ ]:
# Bước: Xây dựng mô hình LSTM
input_shape = (X_class_train.shape[1], 1)

model_lstm= Sequential()
model_lstm.add(LSTM(64, input_shape=input_shape, activation='relu', return_sequences=True))
model_lstm.add(LSTM(32, activation='relu'))
model_lstm.add(Dense(16, activation='relu'))
model_lstm.add(Dense(5, activation='softmax'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_lstm.fit(X_class_train.reshape(X_class_train.shape[0], X_class_train.shape[1], 1), y_class_train, epochs=5, batch_size=32)

In [ ]:
predictions = model_lstm.predict(X_class_test)
y_pred_classes = np.argmax(predictions, axis=1)

##### ĐÁNH GIÁ MÔ HÌNH

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

Accuracy

In [ ]:
report = classification_report(y_class_test, y_pred_classes)
print(report)

Sensitivity

In [ ]:
conf_matrix = confusion_matrix(y_class_test, y_pred_classes)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, y_pred_classes == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, y_pred_classes == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, y_pred_classes == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, y_pred_classes == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, y_pred_classes == 4)

In [ ]:
# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


Precision

In [ ]:
from sklearn.metrics import classification_report
# Lấy báo cáo phân loại cho các lớp
report = classification_report(y_class_test, y_pred_classes)
print(report)


#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Khởi tạo mô hình KNeighborsClassifier với số hàng xóm (neighbors) cần xem xét
knn_model = KNeighborsClassifier(n_neighbors=5)  # Tuỳ chỉnh số hàng xóm nếu cần
knn_model.fit(X_class_train, y_class_train)

In [ ]:
predictions = knn_model.predict(X_class_test)

##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

import matplotlib.pyplot as plt

# Sử dụng recall_score để tính sensitivity cho từng lớp
from sklearn.metrics import recall_score

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Khởi tạo mô hình KNeighborsClassifier với số hàng xóm (neighbors) cần xem xét
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Tuỳ chỉnh số cây (n_estimators) nếu cần

In [ ]:
rf_model.fit(X_class_train, y_class_train)

In [ ]:
predictions = rf_model.predict(X_class_test)

##### Danh gia mo hinh

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Decision Tree

##### Xây dựng mô hình

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Khởi tạo mô hình DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42)

In [ ]:
dt_model.fit(X_class_train, y_class_train)

In [ ]:
predictions = dt_model.predict(X_class_test)

##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### LogisticRegression

##### Xay dung mo hinh

In [ ]:
from sklearn.linear_model import LogisticRegression
# Khởi tạo mô hình Logistic Regression
logistic_model = LogisticRegression()
# Huấn luyện mô hình trên tập huấn luyện
logistic_model.fit(X_class_train, y_class_train)

In [ ]:
predictions = logistic_model.predict(X_class_test)

##### Danh gia mo hinh

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### NaiveBayes

##### Xay dung mo hinh

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Khởi tạo mô hình Naive Bayes (Gaussian Naive Bayes)
nb_model = GaussianNB()
# Huấn luyện mô hình trên tập huấn luyện
nb_model.fit(X_class_train, y_class_train)

In [ ]:
predictions = nb_model.predict(X_class_test)

##### Danh gia mo hinh

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### GradientBoostingClassifier

##### Xay dung mo hinh

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=3)
# Đào tạo mô hình Gradient Boosting trên dữ liệu huấn luyện
gb_model.fit(X_class_train, y_class_train)

In [ ]:
# Dự đoán với mô hình đã đào tạo
predictions = gb_model.predict(X_class_test)

##### Danh gia mo hinh

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Mô hình SVM

In [ ]:
from sklearn.svm import SVC

# Khởi tạo mô hình SVM
svm_model = SVC(random_state=42)
svm_model.fit(X_class_train, y_class_train)
predictions = svm_model.predict(X_class_test)


##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Mô hình Multi-layer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

# Khởi tạo mô hình MLP
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_class_train, y_class_train)
predictions = mlp_model.predict(X_class_test)


##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Mô hình BiLSTM

In [ ]:
from tensorflow.keras.layers import Bidirectional

# Bước: Xây dựng mô hình BiLSTM
input_shape = (X_class_train.shape[1], 1)

model_bilstm = Sequential()
model_bilstm.add(Bidirectional(LSTM(64, input_shape=input_shape, activation='relu', return_sequences=True)))
model_bilstm.add(Bidirectional(LSTM(32, activation='relu')))
model_bilstm.add(Dense(16, activation='relu'))
model_bilstm.add(Dense(5, activation='softmax'))
model_bilstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bilstm.fit(X_class_train.reshape(X_class_train.shape[0], X_class_train.shape[1], 1), y_class_train, epochs=5, batch_size=32)


In [ ]:
predictions_bilstm = model_bilstm.predict(X_class_test)
y_pred_classes_bilstm = np.argmax(predictions_bilstm, axis=1)

##### Đánh giá mô hình 

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_class_test, y_pred_classes_bilstm)
print(report)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, y_pred_classes_bilstm)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, y_pred_classes_bilstm == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, y_pred_classes_bilstm == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, y_pred_classes_bilstm == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, y_pred_classes_bilstm == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, y_pred_classes_bilstm == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Mô hình ANN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Bước: Xây dựng mô hình ANN
input_shape = X_class_train.shape[1]  # Số lượng đặc trưng đầu vào

model_ann = Sequential()
model_ann.add(Dense(64, input_shape=(input_shape,), activation='relu'))
model_ann.add(Dense(32, activation='relu'))
model_ann.add(Dense(16, activation='relu'))
model_ann.add(Dense(5, activation='softmax'))

model_ann.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ann.fit(X_class_train, y_class_train, epochs=10, batch_size=32)
predictions_ann = model_ann.predict(X_class_test)
predictions = np.argmax(predictions_ann, axis=1)

##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Mô hình CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten

# Bước: Xây dựng mô hình CNN từ đoạn code LSTM
input_shape = (X_class_train.shape[1], 1)

model_cnn = Sequential()
model_cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(LSTM(64, activation='relu', return_sequences=True))
model_cnn.add(LSTM(32, activation='relu'))
model_cnn.add(Dense(16, activation='relu'))
model_cnn.add(Dense(5, activation='softmax'))

model_cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.fit(X_class_train, y_class_train, epochs=5, batch_size=32)


In [ ]:
predictions = model_cnn.predict(X_class_test)
predictions = np.argmax(predictions, axis=1)

##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


#### Mô hình RNN

In [ ]:
from tensorflow.keras.layers import SimpleRNN

# Bước: Xây dựng mô hình RNN dựa trên mô hình LSTM đã có
input_shape = (X_class_train.shape[1], 1)

model_rnn = Sequential()
model_rnn.add(LSTM(64, input_shape=input_shape, activation='relu', return_sequences=True))
model_rnn.add(SimpleRNN(32, activation='relu'))
model_rnn.add(Dense(16, activation='relu'))
model_rnn.add(Dense(5, activation='softmax'))

model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.fit(X_class_train, y_class_train, epochs=5, batch_size=32)

In [ ]:
predictions_rnn = model_rnn.predict(X_class_test)
predictions = np.argmax(predictions_rnn, axis=1)

##### Đánh giá mô hình

In [ ]:
report = classification_report(y_class_test, predictions)
print(report)

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()


##### Đánh giá

In [ ]:
# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_class_test, predictions)
# Tạo biểu đồ dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sensitivity_class_0 = recall_score(y_class_test == 0, predictions == 0)
sensitivity_class_1 = recall_score(y_class_test == 1, predictions == 1)
sensitivity_class_2 = recall_score(y_class_test == 2, predictions == 2)
sensitivity_class_3 = recall_score(y_class_test == 3, predictions == 3)
sensitivity_class_4 = recall_score(y_class_test == 4, predictions == 4)

# Giả sử bạn đã tính được sensitivity cho các lớp
sensitivity_scores = [sensitivity_class_0,sensitivity_class_1, sensitivity_class_2, sensitivity_class_3, sensitivity_class_4]

# Tên của các lớp
class_names = ['Class 0','Class 1', 'Class 2', 'Class 3', 'Class 4']

# Vẽ biểu đồ cột
plt.figure(figsize=(8, 6))
plt.bar(class_names, sensitivity_scores, color='skyblue')
plt.title('Sensitivity for Each Class')
plt.xlabel('Class')
plt.ylabel('Sensitivity')
plt.ylim(0, 1)  # Đặt giới hạn trục y từ 0 đến 1 để hiển thị độ nhạy từ 0% đến 100%
plt.show()
